In [ ]:
import pandas as pd
import os
import sys

In [ ]:
# set directory to project root
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

In [ ]:
head_path = "\data\Head\misclassifications_head.json"
neckHead_path = "\data\NeckHead\misclassifications_neckHead.json"
entireModel_path = "\data\EntireModel\misclassifications_entireModel.json"

In [ ]:

miscalssifications_head = pd.read_json(head_path)